In [23]:
#####ノイズあり対応トピックモデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import gensim
import itertools
from numpy.random import *
from scipy import optimize
from scipy import sparse
import seaborn as sns

In [24]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [25]:
####データの発生####
##データの設定
#文書の設定
k = 15   #トピック数
d = 5000   #文書数
v  = 1200   #語彙数
w = np.random.poisson(np.random.gamma(65, 1/0.5, d), d)   #1文書あたりの単語数
a1 = 150   #トピックに関係のあるタグ数
a2 = 100   #トピックに関係のないタグ数
a = a1 + a2
x = np.random.poisson(40, d)
x[x < 2] = 3
f1 = np.sum(w)
f2 = np.sum(x)

#ベクトルの連番の設定
vec_f1 = np.array(range(f1))
vec_f2 = np.array(range(f2))
vec_k = np.array(range(k))
vec_v = np.array(range(v))
vec_a = np.array(range(a))

In [26]:
##IDとインデックスの設定
#IDの設定
d_id1 = np.repeat(range(d), w)
d_id2 = np.repeat(range(d), x)

#インデックスの設定
index1 = np.array(range(f1))
index2 = np.array(range(f2))
d_list1 = [i for i in range(d)]
d_list2 = [i for i in range(d)]
for i in range(d):
    d_list1[i] = index1[d_id1==i]
    d_list2[i] = index2[d_id2==i]
d_dt1 = sparse.coo_matrix((np.repeat(1, f1), (range(f1), d_id1)), shape=(f1, d)).tocsr() 
d_dt2 = sparse.coo_matrix((np.repeat(1, f2), (range(f2), d_id2)), shape=(f2, d)).tocsr() 
d_dt_T1 = d_dt1.T
d_dt_T2 = d_dt2.T

In [27]:
##パラメータの設定
#ディレクリ事前分布のパラメータを設定
alpha0 = np.repeat(0.15, k)   #文書のディレクリ事前分布のパラメータ
alpha1 = np.repeat(0.05, v)   #単語のディレクリ事前分布のパラメータ
alpha2 = np.hstack((np.repeat(0.1, a1), np.repeat(0.001, a2)))   #トピックに関係のあるタグのディレクリ事前分布のパラメータ
alpha3 = np.hstack((np.repeat(0.001, a1), np.repeat(1.25, a2)))   #トピックに関係のないタグのディリクレ事前分布のパラメータ
beta0 = betat0 = np.random.beta(5.0, 2.0, d)

In [28]:
##すべての単語が出現するまで生成を繰り返す
rp = 0
while True:
    rp = rp + 1
    print(rp)
    
    #ディリクレ事前分布からパラメータを生成
    theta = thetat = np.random.dirichlet(alpha0, d)   #文書ごとのトピック分布
    phi = np.random.dirichlet(alpha1, k)   #トピックごとの単語分布
    omega = omegat = np.random.dirichlet(alpha2, k)   #トピックに関係のある補助情報分布
    gamma = gammat = np.random.dirichlet(alpha3, 1)   #トピックに関係のない補助情報分布
    omega0 = np.vstack((omega, gamma))   #補助情報分布の結合
    
    #出現確率が低い単語を入れ替える
    index_v = np.array(range(v))[np.max(phi, axis=0) <= (k*5)/f1]
    for j in range(index_v.shape[0]):
        phi[np.argmax(np.random.multinomial(1, np.repeat(1/k, k), 1)), index_v[j]] = (k*5)/f1
    phit = phi = phi / np.sum(phi, axis=1).reshape(k, 1)
    
    #データの格納用配列
    WX = np.zeros((d, v))
    AX = np.zeros((d, a))
    wd = np.zeros(f1, dtype="int")
    ad = np.zeros(f2, dtype="int") 
    Y = np.repeat(0, f2)
    Z1 = np.zeros((f1, k), dtype="int")
    Z2 = np.zeros((f2, k), dtype="int")
    

    ##文書ごとにトピックと単語を生成する
    for i in range(d):
        
        #文書のトピックを生成
        z1 = np.random.multinomial(1, theta[i, :], w[i])
        z1_vec = np.dot(z1, vec_k)

        #文書のトピック分布から単語を生成
        word = np.array(rmnom(phi[z1_vec, ], w[i], v, np.arange(w[i])).todense())
        WX[i, :] = np.sum(word, axis=0)

        #ベルヌーイ分布からトピックに関係があるかどうかを生成
        y_vec = np.random.binomial(1, beta0[i], x[i])

        #文書のトピック分布から補助情報トピックを生成
        z2_aux = np.random.multinomial(1, theta[i, :], x[i])
        z2 = np.concatenate((z2_aux * y_vec.reshape((x[i], 1)),1-y_vec.reshape((x[i], 1))), axis=1)
        z2_vec = np.dot(z2, np.hstack((vec_k, k)))

        #生成したトピックの単語分布に従い単語を生成
        aux = np.array(rmnom(omega0[z2_vec, ], x[i], a, np.arange(x[i])).todense())
        AX[i, :] = np.sum(aux, axis=0)

        #文書トピックおよび補助情報トピックを格納
        Y[d_list2[i]] = y_vec
        Z1[d_list1[i], ] = z1
        Z2[d_list2[i], ] = z2[:, np.arange(k)]
        wd[d_list1[i]] = np.dot(word, np.arange(v))
        ad[d_list2[i]] = np.dot(aux, np.arange(a))

    print(np.min(np.sum(WX, axis=0)))
    if (np.min(np.sum(WX, axis=0)) > 0.0) & (np.min(np.sum(AX, axis=0)) > 0.0):
        break

1
3.0


In [29]:
#単語のインデックスとスパース行列を作成
index1 = np.array(range(f1))
index2 = np.array(range(f2))
w_list = [j for j in range(v)]
a_list = [j for j in range(a)]
for j in range(v):
    w_list[j] = index1[wd==j]
for j in range(a):
    a_list[j] = index2[ad==j]
sparse_data1 = sparse.coo_matrix((np.repeat(1, f1), (range(f1), wd)), shape=(f1, v)).tocsr()   
sparse_data2 = sparse.coo_matrix((np.repeat(1, f2), (range(f2), ad)), shape=(f2, a)).tocsr()   
sparse_data_T1 = sparse_data1.T
sparse_data_T2 = sparse_data2.T
aux_data = np.array(sparse_data2.todense())

In [30]:
#トピック尤度と負担率を計算する関数
def LLho(theta, phi, d_id, wd, f, k):
    Lho = theta[d_id, ] * (phi.T)[wd, ]
    topic_rate = Lho / np.sum(Lho, axis=1).reshape(f, 1)
    return Lho, topic_rate

In [31]:
##アルゴリズムの設定
R = 3000   #サンプリング回数
keep = 2   #2回に1回の割合でサンプリング結果を格納
disp = 10
iter = 0
burnin = int(500/keep)

##事前分布の設定
alpha01 = 0.1
alpha02 = 0.1
beta01 = 0.1
beta02 = 0.1
gamma01 = 0.25
gamma02 = 0.25

##パラメータの真値
theta = thetat 
phi = phit
omega = omegat
gamma = gammat.reshape((a))
r = betat0

##パラメータの初期値
theta = np.random.dirichlet(np.repeat(1, k), d)   #文書トピックのパラメータの初期値
phi = np.random.dirichlet(np.repeat(2.0, v), k)   #単語分布のパラメータの初期値
omega = np.random.dirichlet(np.repeat(10, a), k)   #内容に関係のあるタグ分布のパラメータの初期値
gamma = np.random.dirichlet(np.repeat(10, a), 1).reshape((a))   #内容に関係のないタグ分布のパラメータの初期値
r = np.repeat(0.5, d)   #内容に関係があるかどうかの混合率

##パラメータの格納用配列
THETA = np.zeros((d, k, R//keep), dtype='float')
PHI = np.zeros((k, v, R//keep), dtype='float')
OMEGA = np.zeros((k, a, R//keep), dtype='float')
GAMMA = np.zeros((R//keep, a), dtype='float')
LAMBDA = np.zeros((R//keep, d), dtype='float')
S_SEG = np.zeros(f2, dtype='int')
W_SEG = np.zeros((f1, k), dtype='int')
A_SEG = np.zeros((f2, k), dtype='int')

##対数尤度の基準値
#ユニグラムモデルの対数尤度
LLst1 = np.sum(np.log((np.sum(WX, axis=0) / f1)[wd]))
LLst2 = np.sum(np.log((np.sum(AX, axis=0) / f2)[ad]))
LLst = LLst1 + LLst2
print(np.round(np.array((LLst, LLst1, LLst2)), 1))

#真値での対数尤度
LLbest1 = np.sum(np.log(np.sum(thetat[d_id1, ] * (phit.T)[wd, ], axis=1)))
LLbest2 = np.sum(np.log(np.sum((thetat[d_id2, ] * (omegat.T)[ad, ])[Y==1, ], axis=1))) + \
              np.sum(np.log(((1-betat0[d_id2]) * (gammat.reshape(-1))[ad])[Y==0, ]))
LLbest = LLbest1 + LLbest2
print(np.round(np.array((LLbest, LLbest1, LLbest2)), 1))

[-5279057.2 -4241294.7 -1037762.6]
[-4573539.4 -3685247.7  -888291.7]


In [32]:
####ノイズあり対応トピックモデルをベイズ推定####
for rp in range(R): 

    ##単語トピックを生成
    #トピック選択確率を設定
    out_word = LLho(theta, phi, d_id1, wd, f1, k)    
    topic_rate = out_word[1]

    #多項分布からトピックを生成
    Zi1 = rmnom(topic_rate, f1, k, vec_f1).tocsr()

    ##単語トピックのパラメータをサンプリング
    #トピック分布をサンプリング
    wsum = np.array(np.dot(d_dt_T1, Zi1).todense()) + alpha01
    for i in range(d):
        theta[i, :] = np.random.dirichlet(wsum[i, ], 1)

    #単語分布をサンプリング
    vsum = np.array(np.dot(sparse_data_T1, Zi1).T.todense()) + beta01
    for j in range(k):
        phi[j, :] = np.random.dirichlet(vsum[j, ], 1)

    ##補助情報がトピックに関係あるかどうかをサンプリング
    #ベルヌーイ分布の確率を設定
    r_vec = r[d_id2]   #混合率のベクトル
    theta_aux = (wsum - alpha01) / w.reshape(d, 1)   #観測トピックからタグのトピック分布を決定  
    tau01 = r_vec * np.sum(theta_aux[d_id2, :] * omega.T[ad, :], axis=1)   #補助変数トピックの尤度
    tau02 = (1-r_vec) * gamma[ad]   #ノイズ補助変数の尤度
    tau1 = tau01 / (tau01 + tau02)   #潜在変数の割当確率

    #ベルヌーイ分布から潜在変数をサンプリング
    S = numpy.random.binomial(1, tau1, f2)
    index_s = np.arange(f2)[S==1]   #トピックに関係のあるタグのインデックス

    #ベータ分布より混合率の期待値をサンプリング
    #ベータ分布のパラメータ
    beta = np.zeros((d, 2))
    for i in range(d):
        n = np.sum(S[d_list2[i]])
        beta[i, 0] = n + gamma01
        beta[i, 1] = x[i] - n + gamma02

    #ベータ分布より混合率をサンプリング
    r = numpy.random.beta(beta[:, 0], beta[:, 1], d)

    ##補助情報トピックを生成
    #タグごとにトピックの出現確率を設定
    n = np.sum(S) 
    out_aux = LLho(theta_aux, omega, d_id2, ad, f2, k)
    aux_rate = out_aux[1][index_s, ]

    #多項分布からトピックを生成
    Zi2 = np.zeros((f2, k))
    Zi2[index_s, ] = rmnom(aux_rate, n, k, np.arange(n)).todense()
    Zi2 = sparse.csr_matrix(Zi2)

    ##補助情報トピックのパラメータをサンプリング
    #単語分布をサンプリング
    af = np.array(np.dot(sparse_data_T2, Zi2).T.todense()) + gamma01
    for j in range(k):
        omega[j, :] = np.random.dirichlet(af[j, ], 1)

    #内容に関係のない補助情報分布omegaをサンプリング
    nf = np.sum(aux_data * (1-S).reshape(f2, 1), axis=0) + gamma02   #ディリクレ分布分布のパラメータ
    gamma = numpy.random.dirichlet(nf, 1).reshape(-1)

    ##パラメータの格納とサンプリング結果の表示
    #サンプリング結果の格納
    if rp%keep==0:
        mkeep = rp//keep
        THETA[:, :, mkeep] = theta
        PHI[:, :, mkeep] = phi
        OMEGA[:, :, mkeep] = omega
        GAMMA[mkeep, :] = gamma
        LAMBDA[mkeep, :] = r

    #トピック割当はバーンイン期間を超えたら格納
    if rp%keep==0 & rp >= burnin:
        W_SEG = W_SEG + Zi1
        A_SEG = A_SEG + Zi2
        S_SEG = S_SEG + S 

    if rp%disp==0:
        #対数尤度の更新
        LL1 = np.sum(np.log(np.sum(theta[d_id1, ] * (phi.T)[wd, ], axis=1)))
        LL2 = np.sum(np.log(np.sum((theta[d_id2, ] * (omega.T)[ad, ])[S==1, ], axis=1))) + \
                  np.sum(np.log(((1-beta0[d_id2]) * (gamma.reshape(-1))[ad])[S==0, ]))
        LL = LL1 + LL2

        #サンプリング結果を確認
        print(rp)
        print(np.round(np.vstack((np.array([LL, LL1, LL2]), np.array([LLst, LLst1, LLst2]), np.array([LLbest, LLbest1, LLbest2]))), 1))
        print(np.round(np.vstack((gamma[(a1-6):(a1+6)], (gammat.reshape(-1))[(a1-6):(a1+6)])), 3))

0
[[-5424283.1 -4246923.1 -1177359.9]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.011 0.001 0.007 0.003 0.002 0.009 0.    0.003 0.005 0.001 0.005 0.   ]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
10
[[-5241600.7 -4094191.2 -1147409.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.012 0.001 0.007 0.001 0.002 0.01  0.001 0.004 0.004 0.001 0.003 0.   ]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
20
[[-4979580.2 -3913379.7 -1066200.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.01  0.    0.008 0.001 0.002 0.01  0.001 0.005 0.006 0.001 0.003 0.   ]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
30
[[-4829416.2 -3820900.4 -1008515.8]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.005 0.    0.006 0.001 0.002 0.008 0.001 0.007 0.009 0.001 0.004 0.001]
 [0.    0.    0.    0.   

320
[[-4615361.9 -3686311.6  -929050.4]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.002 0.    0.    0.    0.    0.001 0.001 0.011 0.018 0.002 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
330
[[-4614662.4 -3686072.   -928590.4]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.001 0.003 0.01  0.016 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
340
[[-4614812.1 -3686348.1  -928464.1]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.    0.001 0.011 0.017 0.002 0.007 0.002]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
350
[[-4614883.9 -3686627.5  -928256.4]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.001 0.002 0.01  0.018 0.001 0.006 0.001]
 [0.    0.    0.    

640
[[-4615104.4 -3686358.9  -928745.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.001 0.002 0.009 0.018 0.002 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
650
[[-4615294.2 -3686156.1  -929138.1]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.001 0.002 0.01  0.017 0.002 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
660
[[-4615527.1 -3686270.5  -929256.6]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.    0.002 0.009 0.017 0.002 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
670
[[-4614867.2 -3686158.7  -928708.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.    0.002 0.01  0.018 0.002 0.006 0.001]
 [0.    0.    0.    

960
[[-4613347.5 -3686052.7  -927294.7]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.002 0.01  0.017 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
970
[[-4614736.  -3686435.4  -928300.6]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.001 0.01  0.019 0.002 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
980
[[-4614213.5 -3686191.8  -928021.7]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.002 0.011 0.018 0.002 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
990
[[-4614667.8 -3686298.2  -928369.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.002 0.01  0.019 0.001 0.007 0.001]
 [0.    0.    0.    

1270
[[-4614842.9 -3686119.3  -928723.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.001 0.011 0.018 0.002 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1280
[[-4614409.8 -3686022.2  -928387.6]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.002 0.01  0.017 0.001 0.007 0.002]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1290
[[-4613934.6 -3686111.7  -927822.9]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.001 0.    0.001 0.002 0.011 0.017 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1300
[[-4615074.3 -3686154.3  -928920. ]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.002 0.011 0.018 0.001 0.007 0.001]
 [0.    0.    0.

1580
[[-4616908.1 -3686571.9  -930336.2]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.001 0.011 0.018 0.002 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1590
[[-4615795.2 -3686380.6  -929414.7]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.    0.002 0.01  0.018 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1600
[[-4614983.5 -3686154.   -928829.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.001 0.    0.    0.    0.001 0.01  0.019 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1610
[[-4615652.4 -3686165.1  -929487.4]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.001 0.001 0.01  0.017 0.001 0.007 0.001]
 [0.    0.    0.

1890
[[-4614289.6 -3686228.3  -928061.3]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.001 0.    0.    0.    0.002 0.011 0.017 0.001 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1900
[[-4614717.8 -3686552.3  -928165.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.002 0.01  0.017 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1910
[[-4614004.7 -3686337.1  -927667.6]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.001 0.002 0.009 0.018 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
1920
[[-4614221.1 -3686230.3  -927990.8]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.002 0.011 0.018 0.001 0.006 0.001]
 [0.    0.    0.

2200
[[-4613213.1 -3686246.9  -926966.2]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.002 0.01  0.017 0.002 0.008 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2210
[[-4613971.4 -3686357.6  -927613.8]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.001 0.    0.    0.    0.001 0.011 0.017 0.001 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2220
[[-4614158.9 -3686407.2  -927751.7]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.001 0.009 0.018 0.002 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2230
[[-4612834.1 -3686095.7  -926738.3]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.    0.002 0.011 0.018 0.001 0.006 0.001]
 [0.    0.    0.

2510
[[-4615126.3 -3686490.8  -928635.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.001 0.001 0.01  0.016 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2520
[[-4614362.5 -3686133.2  -928229.3]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.    0.002 0.011 0.019 0.001 0.006 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2530
[[-4614045.1 -3686336.2  -927708.9]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.001 0.    0.    0.001 0.002 0.011 0.017 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2540
[[-4613797.9 -3686080.5  -927717.4]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.001 0.    0.002 0.011 0.018 0.001 0.006 0.001]
 [0.    0.    0.

2820
[[-4614062.1 -3686246.9  -927815.3]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.    0.    0.    0.    0.    0.001 0.002 0.012 0.018 0.002 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2830
[[-4615557.4 -3686269.9  -929287.5]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.001 0.001 0.001 0.011 0.018 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2840
[[-4614996.7 -3685983.4  -929013.2]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.    0.    0.    0.    0.001 0.011 0.017 0.001 0.007 0.001]
 [0.    0.    0.    0.    0.    0.    0.002 0.012 0.019 0.002 0.007 0.001]]
2850
[[-4615240.2 -3686083.   -929157.2]
 [-5279057.2 -4241294.7 -1037762.6]
 [-4573539.4 -3685247.7  -888291.7]]
[[0.001 0.    0.001 0.    0.    0.    0.001 0.01  0.017 0.001 0.006 0.001]
 [0.    0.    0.